In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import api_key
c = Census(api_key, year=2017)

In [2]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_df = pd.DataFrame(census_data)
'''

census_data = c.acs5.get(('NAME', "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"),
          {'for': 'state:{}'.format(states.MD.fips)})'''

# Column Reordering
census_df = census_df.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})



In [6]:
# Add in Poverty Rate (Poverty Count / Population)
census_df["Poverty Rate"] = 100 * \
    census_df["Poverty Count"].astype(
        int) / census_df["Population"].astype(int)

# Final DataFrame
census_df = census_df[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]

census_df["Zipcode"] = census_df["Zipcode"].astype(str)
census_df.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,00601,17599.0,38.9,11757.0,7041.0,11282.0,64.105915
1,00602,39209.0,40.9,16190.0,8978.0,20428.0,52.100283
2,00603,50135.0,40.4,16645.0,10897.0,25176.0,50.216416
3,00606,6304.0,42.8,13387.0,5960.0,4092.0,64.911168
4,00610,27590.0,41.4,18741.0,9266.0,12553.0,45.498369


In [7]:
zip_list = ['55418', '55411', '55405', '55401', '55408', '55414', '55403',
       '55402', '55415', '55404', '55412', '55454', '55406', '55413',
       '55409', '55416', '55419', '55407', '55455', '55422', '55417',
       '55410', '55430', '55487', '55114', '55450', '55421', '55423',
       '55429']   #Hard coded....can get from ID_to_Zipcode program

In [14]:
zipcode_df = pd.DataFrame({"Zipcode": zip_list})


census_mpls_df = pd.merge(zipcode_df, census_df, on = 'Zipcode', how="left")

In [15]:
census_mpls_df.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,55418,30843.0,35.0,64180.0,34162.0,3932.0,12.748436
1,55411,30917.0,26.3,36512.0,16087.0,9682.0,31.316104
2,55405,15976.0,31.9,56365.0,46433.0,2277.0,14.252629
3,55401,8798.0,34.1,90025.0,78295.0,1251.0,14.219141
4,55408,31313.0,30.3,52466.0,37004.0,4884.0,15.597356
